In [1]:
%pip install semantic-link-labs

StatementMeta(, a272cdad-9639-4d06-9e03-cc19544753e7, 7, Finished, Available, Finished)

  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... - done
  Installing backend dependencies ... - \ done
  Preparing metadata (pyproject.toml) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 580.3/580.3 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 128.2 MB/s eta 0:00:00
  Created wheel for jsonpath-ng: filename=jsonpath_ng-1.7.0-py3-none-any.whl size=30099 sha256=c9b1e7080a017862195903ef0ae982a1ef31619198625ab327141e4d41dfe2ca
  Stored in directory: /home/trusted-service-user/.cache/pip/wheels/6b/47/94/d8ae7a868e6642221f5caa53d6aedce4ff138c76c9f031a3da
Successfully built jsonpath-ng
  Attempting uninstall: semantic-link-sempy
    Found existing installation: semantic-link-sempy 0.7.7
    Not uninstalling semantic-link-sempy at /hom

In [2]:
import sempy_labs as labs
from sempy_labs.directlake._get_shared_expression import get_shared_expression
from sempy_labs.tom import connect_semantic_model
from sempy_labs import report
import json

StatementMeta(, a272cdad-9639-4d06-9e03-cc19544753e7, 9, Finished, Available, Finished)

In [3]:
lakehouse_name  =  "FabricDocs"
lakehouse  =  mssparkutils.lakehouse.get(lakehouse_name)
abfsPath = lakehouse.properties["abfsPath"]
workspace_name  =  notebookutils.runtime.context.get("currentWorkspaceName") # Get current workspace name

StatementMeta(, a272cdad-9639-4d06-9e03-cc19544753e7, 10, Finished, Available, Finished)

In [4]:
### Create new semantic model
semantic_model_name  =  f"{lakehouse_name}_Model"
labs.create_blank_semantic_model(semantic_model_name)

StatementMeta(, a272cdad-9639-4d06-9e03-cc19544753e7, 11, Finished, Available, Finished)

🟢 The 'FabricDocs_Model' semantic model was created within the 'PG - Fabric REST Api Docs' workspace.


In [5]:
### Add tables, columns, hierarchies and measures
lakehouse_workspace = lakehouse.workspaceId
shEx = get_shared_expression(lakehouse_name, lakehouse_workspace)

tables = {
    "FabricRestApiDocs": {
        "display_name": "Fabric REST APIs",
        "columns": {
            "Title": {"source_column": "Title", "data_type": "string" },
            "Service": {"source_column": "Service", "data_type": "string" },
            "API": {"source_column": "API", "data_type": "string" },
            "Article Url": {"source_column": "ArticleUrl", "data_type": "string", "data_category": "WebURL"},
            "Supports User": {"source_column": "SupportUserIdentity", "data_type": "string" },
            "Supports Service principal": {"source_column": "SupportSPN", "data_type": "string" },
            "Supports Managed identities": {"source_column": "SupportMI", "data_type": "string" }
        },
        "hierarchies" : {
            "Service hierarchy": {"columns": ["Service", "API", "Title"] }
        }

    },
    "SupportedIdentities": {
        "display_name": "Supported Identities",
        "columns": {
            "IdentityNo": {"source_column": "IdentityNo", "data_type": "integer", "hidden": True },
            "Identity": {"source_column": "Identity", "data_type": "string", "sort_by" : "IdentityNo" },
            "Identity Description": {"source_column": "Identity_desc", "data_type": "string" },
            "Identity Support Article Url": {"source_column": "IdentitySupportArticle", "data_type": "string", "data_category": "WebURL" }
        }
    }
}

with connect_semantic_model(dataset=semantic_model_name, readonly=False, workspace=workspace_name) as tom:
    if not any(e.Name == "DatabaseQuery" for e in tom.model.Expressions):
        tom.add_expression("DatabaseQuery", expression=shEx)
        print(f"The 'DatabaseQuery' expression has been added.")

    for table_name, table_data in tables.items():
        display_name = table_data["display_name"]
        columns = table_data.get("columns", None)
        hierarchies = table_data.get("hierarchies", None)
        
        print(f"Processing table: {table_name} ({display_name})")
        if not any(t.Name == display_name for t in tom.model.Tables):
            tom.add_table(name = display_name)
            tom.add_entity_partition(table_name = display_name, entity_name = table_name)
            print(f"The '{display_name}' table has been added.")
        
        for column_name, attributes in columns.items():
            source_column = attributes["source_column"]
            data_type = attributes["data_type"]
            is_hidden = attributes.get("hidden", False)
            data_category = attributes.get("data_category", None)
            sort_by = attributes.get("sort_by", None)
            
            if not any(c.Name == column_name and c.Parent.Name == display_name for c in tom.all_columns()):
                tom.add_data_column(
                    table_name = display_name,
                    column_name = column_name,
                    source_column = source_column,
                    hidden = is_hidden,
                    data_type = data_type,
                    data_category = data_category,
                )
                if(sort_by is not None):
                    tom.set_sort_by_column(table_name = display_name, column_name = column_name, sort_by_column = sort_by)
            
                print(f"The '{source_column}' source column has been added as column '{column_name}'")



        if hierarchies is not None:
            for hierarchy_name, hierarchy_columns in hierarchies.items():
                if not any(h.Name == hierarchy_name for h in tom.all_hierarchies()):
                    h_columns = hierarchy_columns["columns"]
                    tom.add_hierarchy(
                        table_name = display_name,
                        hierarchy_name = hierarchy_name,
                        columns = h_columns
    )

    # Adding measures
    m_expression = """SWITCH(
                        SELECTEDVALUE( 'Supported Identities'[IdentityNo] ),
                        1, CALCULATE( COUNTROWS( 'Fabric REST APIs' ), 'Fabric REST APIs'[Supports User] = "Yes" ),
                        2, CALCULATE( COUNTROWS( 'Fabric REST APIs' ), 'Fabric REST APIs'[Supports Service principal] = "Yes" ),
                        3, CALCULATE( COUNTROWS( 'Fabric REST APIs' ), 'Fabric REST APIs'[Supports Managed identities] = "Yes" ),
                        COUNTROWS( 'Fabric REST APIs' )
                )"""

    rest_api_table = tom.model.Tables["Fabric REST APIs"]
    measure_name = "Supported identity count"
    if not any(m.Name == measure_name for m in tom.all_measures()):
        tom.add_measure(
            table_name = rest_api_table.Name, 
            measure_name = measure_name, 
            expression = m_expression
        )
        print(f"The '{measure_name}' measure has been added to table '{rest_api_table.Name}'")
    else:
        tom.update_measure(
            measure_name = measure_name, 
            expression = m_expression
        )
        print(f"The '{measure_name}' measure has been updated in table '{rest_api_table.Name}'")

    m_expression = """VAR api_cnt = COUNTROWS('Fabric REST APIs')
        RETURN
            IF(
                [Supported identity count] - api_cnt = 0, UNICHAR( 9989 ),
                IF( ISBLANK([Supported identity count]), UNICHAR( 10060 ), UNICHAR( 128993 ))
            )"""

    rest_api_table = tom.model.Tables["Supported Identities"]
    measure_name = "Identity is supported"
    if not any(m.Name == measure_name for m in tom.all_measures()):
        tom.add_measure(
            table_name = rest_api_table.Name, 
            measure_name = measure_name, 
            expression = m_expression
        )
        print(f"The '{measure_name}' measure has been added to table '{rest_api_table.Name}'")
    else:
        tom.update_measure(
            measure_name = measure_name, 
            expression = m_expression
        )
        print(f"The '{measure_name}' measure has been updated in table '{rest_api_table.Name}'")

StatementMeta(, a272cdad-9639-4d06-9e03-cc19544753e7, 12, Finished, Available, Finished)

The 'DatabaseQuery' expression has been added.
Processing table: FabricRestApiDocs (Fabric REST APIs)
The 'Fabric REST APIs' table has been added.
The 'Title' source column has been added as column 'Title'
The 'Service' source column has been added as column 'Service'
The 'API' source column has been added as column 'API'
The 'ArticleUrl' source column has been added as column 'Article Url'
The 'SupportUserIdentity' source column has been added as column 'Supports User'
The 'SupportSPN' source column has been added as column 'Supports Service principal'
The 'SupportMI' source column has been added as column 'Supports Managed identities'
Processing table: SupportedIdentities (Supported Identities)
The 'Supported Identities' table has been added.
The 'IdentityNo' source column has been added as column 'IdentityNo'
The 'Identity' source column has been added as column 'Identity'
The 'Identity_desc' source column has been added as column 'Identity Description'
The 'IdentitySupportArticle' 

In [7]:
# Read the file as a DataFrame where each row represents a line in the file. 
# NOTE: Remember to attach the lakehouse containing the report.json file.
df = spark.read.text(f"{abfsPath}/Files/report.json")

# Convert the DataFrame rows (lines) into a single string
json_raw = ''.join(df.rdd.map(lambda row: row[0]).collect())
jobject = json.loads(json_raw)

# Create a new report based on the report.json file located in our Lakehouse
labs.report.create_report_from_reportjson(
    report = "Fabric REST API Docs", 
    dataset = semantic_model_name, 
    report_json = jobject, 
    workspace = workspace_name
    )

StatementMeta(, a272cdad-9639-4d06-9e03-cc19544753e7, 14, Finished, Available, Finished)

🟢 Succesfully created the 'Fabric REST API Docs' report within the 'PG - Fabric REST Api Docs' workspace.
